In [1]:
import os
import pandas as pd 
from glob import glob

import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

import chat_bot as cb
import Database_Handler as dh

ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1385489, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


In [2]:
def Check(list1, list2):
    first_keywords = list(filter(lambda x: x[0] == x[1], list(itertools.product(list1, list2))))
    first_keywords = set(map(lambda x: x[0], first_keywords))
    second_keywords = list(filter(lambda x: x[0] != x[1], list(itertools.product(list1, list2))))
    second_keywords = list(filter(lambda x: abs(len(max(x, key = len)) - len(''.join(re.findall('['+x[0]+']', x[1])).strip())) == 1,
                                  second_keywords))
    second_keywords = set(map(lambda x: max(x, key = len), second_keywords))
    keywords = first_keywords|second_keywords
    k2 = list(filter(lambda idx: abs(len(''.join(re.findall('['+CompareWords(*idx)[0]+']', CompareWords(*idx)[1])).strip())-len(min(idx, key = len))) == 0,
                     list(itertools.combinations(keywords, 2))))
    k2 = set(map(lambda x: min(x, key = len), k2))
    list(map(lambda x: keywords.remove(x), k2))
    return keywords

def CompareWords(word1, word2):
    if len(word1) == len(word2):
        if re.sub(' ','', word1) > re.sub(' ','', word2):
            return word2, word1
        elif re.sub(' ','', word1) < re.sub(' ','', word2):
            return word1, word2
        else:
            return word1, word2
    else:
        return min([word1, word2], key = len), max([word1, word2], key = len)

def Check2(list1, list2):
    first_keywords = list(filter(lambda x: x[0] == x[1], list(itertools.product(list1, list2))))
    first_keywords = set(map(lambda x: x[0], first_keywords))
    second_keywords = list(filter(lambda x: x[0] != x[1], list(itertools.product(list1, list2))))
    second_keywords = list(filter(lambda x: abs(len(CompareWords(*x)[1]) - len(''.join(re.findall('('+CompareWords(*x)[0]+')?', CompareWords(*x)[1])).strip())) == 1,
                                  second_keywords))
    second_keywords = set(map(lambda x: CompareWords(*x)[1], second_keywords))
    keywords = first_keywords|second_keywords
    k2 = list(filter(lambda idx: abs(len(''.join(re.findall('['+CompareWords(*idx)[1]+']',CompareWords(*idx)[0])).strip())-len(min(idx, key = len))) == 0,
                     list(itertools.combinations(keywords, 2))))
    k2 = set(map(lambda x: min(x, key = len), k2))
    list(map(lambda x: keywords.remove(x), k2))
    return keywords

def Compare_Keywords(list1, list2, list3):
    list1 = list(map(lambda x: ' '.join(list(dict.fromkeys(x.split(' ')))).strip(), list1))
    list1 = list(dict.fromkeys(list1 + list(map(lambda y: ''.join(y), list(map(lambda x: x.split(' '), list1))))))
    list2 = list(map(lambda x: ' '.join(list(dict.fromkeys(x.split(' ')))).strip(), list2))
    list2 = list(dict.fromkeys(list2 + list(map(lambda y: ''.join(y), list(map(lambda x: x.split(' '), list2))))))
    list3 = list(map(lambda x: ' '.join(list(dict.fromkeys(x.split(' ')))).strip(), list3))
    list3 = list(dict.fromkeys(list3 + list(map(lambda y: ''.join(y), list(map(lambda x: x.split(' '), list3))))))
    m1 = Check2(Check(list1, list2), Check(list2, list1))
    m2 = Check2(Check(list1, list3), Check(list3, list1))
    m3 = Check2(Check(list2, list3), Check(list3, list2))
    total =m1 | m2 | m3
    xyz2 = list(itertools.combinations(total, 2))
    xyz3 = list(filter(lambda x: abs(len(''.join(re.findall('('+CompareWords(*x)[0]+')',CompareWords(*x)[1])).strip()) - len(CompareWords(*x)[0]))==0, xyz2))
    xyz3_1 = set(map(lambda x:CompareWords(*x), xyz3))
    xyz3 = set(map(lambda x:CompareWords(*x)[0], xyz3))
    xyz3 = set(filter(lambda x: x!='', xyz3))
    list(map(lambda x: total.remove(x), xyz3))
    xyz2 = list(itertools.combinations(total, 2))
    xyz4 = list(filter(lambda x: abs(len(''.join(re.findall('['+CompareWords(*x)[0]+']',CompareWords(*x)[1])).strip()) - len(CompareWords(*x)[0]))==0, xyz2))
    xyz4_1 = set(map(lambda x: CompareWords(*x), xyz4))
    xyz4 = set(map(lambda x: CompareWords(*x)[0], xyz4))
    xyz4 = set(filter(lambda x:x!='', xyz4))
    list(map(lambda x: total.remove(x), xyz4))
    xyz2 = list(itertools.combinations(total, 2))
    xyz5 = set(filter(lambda x: re.findall('('+re.sub(' ','',CompareWords(*x)[0])+')',CompareWords(*x)[1]),xyz2)) 
    lstCheck = list(map(lambda x: (CompareWords(*x)[1], CheckDuplicate1(*x)), xyz3_1 | xyz4_1 | xyz5))
    lstCheck = list(filter(lambda x: x[1]!=None, lstCheck))
    total = list(total)
    if lstCheck:
        for check in lstCheck:
            if check[0] in total:
                total[total.index(check[0])] = check[1]
            else:pass
    total = list(set(total))
    xyz6 = list(itertools.combinations(total, 2))
    xyz6 = list(filter(lambda x: re.findall('('+re.sub(' ', '', CompareWords(*x)[0])+')', CompareWords(*x)[1]), xyz6))
    if xyz6:
        for check in xyz6:
            if check[0] in total:
                total[total.index(check[0])]=check[1]
    return set(total)

def CheckDuplicate1(x1,x2):
    lessWord = CompareWords(x1,x2)[0]
    moreWord = CompareWords(x1,x2)[1]
    out1 = re.findall(lessWord, moreWord)
    if len(out1) > 1:
        return out1[0]
    elif len(out1) == 1:
        if re.search(lessWord, moreWord):
            out2 = re.sub(re.search(lessWord, moreWord).group(), '', moreWord).strip()
            lessWord2 = CompareWords(out2, lessWord)[0]
            moreWord2 = CompareWords(out2, lessWord)[1]
            if re.search(lessWord2, moreWord2):
                return moreWord2
            else:
                return None
        else:
            return None
    else:
        return None

In [3]:
def Run(site):
    if site == 'daum':
        collection = 'newsDaum'
    elif site.lower() == 'naver':
        collection = 'newsNaver'
    mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
    dbname = 'hy_db'
    useDb = dh.Use_Database(mongodb, dbname)
    useCollection = dh.Use_Collection(useDb, collection)

    ext_Keyword_Daum_mecab = glob('./data/Keyword_outcome/mecab-'+site.lower()+'*')
    ext_Keyword_Daum_ckonlpy = glob('./data/Keyword_outcome/ckonlpy-'+site.lower()+'*')
    ext_Keyword_Daum_both = glob('./data/Keyword_outcome/mecab_ckonlpy-'+site.lower()+'*')

    ckonlpyDict = dict()
    for file in ext_Keyword_Daum_ckonlpy:
        ckonlpyDict.update(pickle.load(open(file,'rb')))
    mecabDict = dict()
    for file in ext_Keyword_Daum_mecab:
        mecabDict.update(pickle.load(open(file,'rb')))
    bothDict = dict()
    for file in ext_Keyword_Daum_both:
        bothDict.update(pickle.load(open(file,'rb')))
    print (len(ckonlpyDict))
    print (len(mecabDict))
    print (len(bothDict))

    dataList = useCollection.find({'site':site})
    print (dataList.count())
    dataDict = dict()
    i = 0 
    for idx, data in enumerate(dataList):
        idIs = data['_id']._ObjectId__id.hex()
        if idIs in ckonlpyDict.keys() and idIs in mecabDict.keys() and idIs in bothDict.keys():
            keywords = Compare_Keywords(ckonlpyDict[idIs], mecabDict[idIs], bothDict[idIs])
            dataDict[idIs] = keywords
            i+=1
    pickle.dump(dataDict, open('./data/pre_data/keywords/keywords_'+site+'.pickled','wb'))

    print (len(dataDict))
    return dataDict

In [4]:
naver_out = Run('Naver')
daum_out = Run('daum')

15121
15121
15121
15120
15120
9372
9372
9372
9372
9372


In [5]:
def Stastics(site, keywordsdict):
    if site == 'daum':
        binList = []
        collection = 'newsDaum'
    elif site.lower() == 'naver':
        binList = []
        collection = 'newsNaver'
    mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
    dbname = 'hy_db'
    useDb = dh.Use_Database(mongodb, dbname)
    useCollection = dh.Use_Collection(useDb, collection)
    collection = useCollection.find({'site':site})

    dataDict = dict()
    for data in collection:
        dataDict[data['_id']._ObjectId__id.hex()] = idData = dict()
        idData['category'] = data['category']
        idData['date'] = data['date']
        idData['press'] = data['press']
        idData['number_of_comment'] = data['number_of_comment']
        idData['number_of_crawled_comment'] = data['real_number_of_comment']
        idData['rank'] = data['rank']
        idData['title'] = data['title']
        idData['mainText'] = data['mainText']
        idData['keywords'] = data['keywords']
        idData['extracted_keywords'] = keywordsdict[data['_id']._ObjectId__id.hex()]
    pickle.dump(dataDict, open('./data/pre_data/stastics/for_statistics_'+site+'_from_mongodb.pickled','wb'))
    print (len(dataDict))

In [6]:
Stastics('Naver', naver_out)
Stastics('daum', daum_out)

15120
9372


### 결론
* 완벽하지 않다. 
> 플라세보 효과 -> 세보 효과 등으로 단어를 정확히 추출해내지 못하는 경우가 존재함. 